In [9]:
import classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import pickle

## Load Models

### TF-IDF

In [11]:
with open ("tfidf.pk", 'rb') as fin:
    tfidf = pickle.load(fin)

In [12]:
tfidf

TfidfVectorizer(max_df=5, ngram_range=(1, 3), stop_words='english')

### Word2Vec CBOW

In [14]:
with open ("w2v_cbow.pk", 'rb') as fin:
    cbow = pickle.load(fin)

In [15]:
cbow

array([ 0.08266962, -0.0510025 , -0.14384806, -0.20676684, -0.0150829 ,
       -0.14188373, -0.0498458 , -0.07315213,  0.09609771,  0.27924392,
       -0.13390553,  0.04456788, -0.05224732, -0.01276228,  0.05903243,
       -0.00231755,  0.17616057,  0.15871191,  0.22587109,  0.06615329,
       -0.16587675,  0.11609097, -0.00953996,  0.11492729, -0.04434949,
       -0.13707896, -0.03670287, -0.14368547, -0.10731274,  0.3065551 ,
       -0.02827144,  0.07733524], dtype=float32)

### Word2Vec SkipGram

In [16]:
with open ("w2v_sg.pk", "rb") as fin:
    skg = pickle.load(fin)

In [17]:
skg

array([ 0.10540247,  0.00649637, -0.04158486, -0.0870658 ,  0.04231866,
       -0.02794513, -0.05232182,  0.02245027, -0.07803168,  0.07128473,
       -0.08157565,  0.00467052,  0.05193534,  0.00040218,  0.01184358,
        0.068593  ,  0.142206  ,  0.06369126, -0.04586295,  0.03145951,
       -0.02454194,  0.03166866,  0.08151402, -0.11906867,  0.03597011,
       -0.06184277,  0.00555149, -0.07806904, -0.0264233 ,  0.09112785,
        0.03857002, -0.0733909 ], dtype=float32)

## Importing Data

In [18]:
df = pd.read_csv("processed_train.csv")

In [19]:
df

,Unnamed: 0,id,tweet,subtask_a,subtask_b,subtask_c
0,0,86426,"['ask', 'native', 'american', 'take']",OFF,UNT,NaN
1,1,90194,"['go', 'home', '’', 'drunk', 'maga', 'trump', ...",OFF,TIN,IND
2,2,16820,"['amazon', 'investigating', 'chinese', 'employ...",NOT,NaN,NaN
3,3,62688,"['someone', 'shouldve', 'taken', 'piece', 'shi...",OFF,UNT,NaN
4,4,43605,"['obama', 'wanted', 'liberal', 'amp', 'illegal...",NOT,NaN,NaN
...,...,...,...,...,...,...
13235,13235,95338,"['sometimes', 'get', 'strong', 'vibe', 'people...",OFF,TIN,IND
13236,13236,67210,"['benidorm', '✅', 'creamfields', '✅', 'maga', ...",NOT,NaN,NaN
13237,13237,82921,"['report', 'garbage', 'dont', 'give', 'crap']",OFF,TIN,OTH
13238,13238,27429,['pussy'],OFF,UNT,NaN


### Removing Redundant Axes

In [20]:
df = df.drop(['Unnamed: 0', 'subtask_b', 'subtask_c', 'id'], axis=1)

In [21]:
df

,tweet,subtask_a
0,"['ask', 'native', 'american', 'take']",OFF
1,"['go', 'home', '’', 'drunk', 'maga', 'trump', ...",OFF
2,"['amazon', 'investigating', 'chinese', 'employ...",NOT
3,"['someone', 'shouldve', 'taken', 'piece', 'shi...",OFF
4,"['obama', 'wanted', 'liberal', 'amp', 'illegal...",NOT
...,...,...
13235,"['sometimes', 'get', 'strong', 'vibe', 'people...",OFF
13236,"['benidorm', '✅', 'creamfields', '✅', 'maga', ...",NOT
13237,"['report', 'garbage', 'dont', 'give', 'crap']",OFF
13238,['pussy'],OFF


### Renaming Columns

In [22]:
df = df.rename(columns={'subtask_a': 'Offensive'})

In [23]:
df

,tweet,Offensive
0,"['ask', 'native', 'american', 'take']",OFF
1,"['go', 'home', '’', 'drunk', 'maga', 'trump', ...",OFF
2,"['amazon', 'investigating', 'chinese', 'employ...",NOT
3,"['someone', 'shouldve', 'taken', 'piece', 'shi...",OFF
4,"['obama', 'wanted', 'liberal', 'amp', 'illegal...",NOT
...,...,...
13235,"['sometimes', 'get', 'strong', 'vibe', 'people...",OFF
13236,"['benidorm', '✅', 'creamfields', '✅', 'maga', ...",NOT
13237,"['report', 'garbage', 'dont', 'give', 'crap']",OFF
13238,['pussy'],OFF


### Converting Offensive to Numerical Value

In [24]:
def off(cls):
    if cls =='OFF':
        return 1
    return 0

In [25]:
df["Offensive"] = df["Offensive"].apply(off)

In [26]:
df

,tweet,Offensive
0,"['ask', 'native', 'american', 'take']",1
1,"['go', 'home', '’', 'drunk', 'maga', 'trump', ...",1
2,"['amazon', 'investigating', 'chinese', 'employ...",0
3,"['someone', 'shouldve', 'taken', 'piece', 'shi...",1
4,"['obama', 'wanted', 'liberal', 'amp', 'illegal...",0
...,...,...
13235,"['sometimes', 'get', 'strong', 'vibe', 'people...",1
13236,"['benidorm', '✅', 'creamfields', '✅', 'maga', ...",0
13237,"['report', 'garbage', 'dont', 'give', 'crap']",1
13238,['pussy'],1


In [27]:
df['tweet']

0                    ['ask', 'native', 'american', 'take']
1        ['go', 'home', '’', 'drunk', 'maga', 'trump', ...
2        ['amazon', 'investigating', 'chinese', 'employ...
3        ['someone', 'shouldve', 'taken', 'piece', 'shi...
4        ['obama', 'wanted', 'liberal', 'amp', 'illegal...
                               ...                        
13235    ['sometimes', 'get', 'strong', 'vibe', 'people...
13236    ['benidorm', '✅', 'creamfields', '✅', 'maga', ...
13237        ['report', 'garbage', 'dont', 'give', 'crap']
13238                                            ['pussy']
13239    ['spanishrevenge', 'v', 'justice', 'human', 'r...
Name: tweet, Length: 13240, dtype: object